In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
from opt_helpers import Opt_Helpers
from pandas_datareader import data

In [13]:
"""Define parameters for data request."""

tickers = ['AGG', 'EBND', 'URTH', 'EEM']
data_source = 'yahoo'
start_date = '2014-03-31'
end_date = '2018-03-31'

In [14]:
"""Request data, ensure sorted by ascending date, and normalise."""

panel_data = data.DataReader(tickers, data_source, start_date, end_date)
px_data = panel_data.loc['Adj Close'].dropna().sort_index(ascending=True)
px_data = px_data / px_data.iloc[0]

In [15]:
"""Initialise class with price data."""

opt = Opt_Helpers(px_data)

In [35]:
"""
TEST CASE
Add constraints: Portfolio weights must sum to 100%, minimum ann. return > 3.0%,
Add boundaries: Long only, no short positions.
"""

min_ret = 0.035

cons = (
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
    {'type': 'ineq', 'fun': lambda w: opt.summary_stats(w)['exp_return'] - min_ret}
)

bnds = tuple((0, 1) for x in px_data.columns)

In [36]:
"""Run optimisation."""

opt_res = opt.port_optimisation(opt.max_dr, constraints=cons, bounds=bnds)

Optimization terminated successfully. Success==True after 7 iterations.
Total time: 0.20136809349060059 secs


In [37]:
"""Class attributes store optimised weights."""

opt.weights

,pos_wgt,vol_contrib
AGG,0.738948,0.254442
EBND,0.000679,0.000823
EEM,0.032694,0.113515
URTH,0.227679,0.631220


In [38]:
"""Class attributes store optimisation method."""

opt.opt_method

'max_dr'

In [39]:
"""Class attributes store optimisation summary statistics."""

opt.stats

,summary_stats
exp_return,0.035000
max_drawdown,-0.051268
sharpe_ratio,0.889718
volatility,0.039338
